# Offloading workload from Cloud Pak for Data to Watson ML Accelerator

Using the Watson Machine Learning API Python client library (https://github.ibm.com/WML/python-client). You will learn to download a sample Pytorch MNIST model and train it using Watson Machine Learning Accelerator (WMLA) to identify numbers based on images. Then, you will save and deploy the model from your workspace and again use WML to score it. Lastly you will clean up by deleting your deployment. This notebook runs on Python 3.6.



![options](https://github.com/IBM/wmla-learning-path/raw/master/shared-images/CPD-WMLA_3_pythonclient.png)




## Table of Contents:

1. [Setup](#setup)<br>
    1.1 [Initialize python client](#Initialize-python-client)<br>
    1.2 [Set default space](#Set-default-space)<br>
    1.3 [Create library](#Create-library)<br>
2. [Train the model](#train)<br>
    2.1 [Create training](#Create-training)<br>
    2.2 [Monitor training](#Monitor-training)<br>
3. [Save and deploy the model](#deploy)<br>
    3.1 [Save Model to project](#Save-Model-to-project)<br>
    3.2 [Create Deployment](#Create-Deployment)<br>
4. [Scoring the model](#score)<br>
    4.1 [Score deployment](#Score-deployment)<br>
    4.2 [Prediction accuracy](#Prediction-accuracy)<br>
    4.3 [Interact with wider project](#Interact-with-wider-project)<br>
5. [Clean up resources](#Clean-up-resources)

<a id = "setup"></a>
## Setup

Uncomment and run this cell if Python v4 client is not installed.

In [1]:
#!pip install --upgrade watson-machine-learning-client-V4

### Initialize python client

In [1]:
import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

Fill in username and password to access WML in cell below, plus URL for local CPD cluster with WML:

In [2]:
# @hidden_cell

# Authenticate using token for a user which exists on both the CPD and WMLA cluster.
# Otherwise specify username and password

wml_credentials = {
#     "token": token,
    'username': '', # Enter username 
    'password': '', # Enter password 
    "instance_id" : "wml_local",
    "url": "", # Enter URL 
    "version": "2.5.0"
}

In [3]:
# This can take time to run - be patient :)

from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
client.version

'1.0.55'

### Set default space

Set default space for CPD usage (if needed, note that this unsets the project id). Change cell below to be of type `code` to use.

In [4]:
meta_props = {
    client.spaces.ConfigurationMetaNames.NAME: "Pytorch deployment test space MQ"
}

space = client.spaces.store(meta_props)
space_id = client.spaces.get_uid(space)

print("Space id: {}".format(space_id))

client.set.default_space(space_id)

Space id: 07b32a16-b82e-443e-96ae-fab32da0f1b1


'SUCCESS'

In [5]:
client.spaces.list()

------------------------------------  ---------------------------------  ------------------------
GUID                                  NAME                               CREATED
07b32a16-b82e-443e-96ae-fab32da0f1b1  Pytorch deployment test space MQ   2020-07-10T14:41:04.060Z
3d9a4805-1671-4ede-9808-6dfc7a7cfc3e  Keras LSTM model deployment space  2020-07-07T09:17:56.501Z
06ed0262-1720-4867-9f98-974b92926ae8  Kelvin                             2020-06-04T18:46:15.951Z
2d1218ba-d774-4386-a6b1-ace31155361f  CPD WMLA demo space                2020-06-04T14:36:32.519Z
71525c1e-75d7-4155-be31-17d0984cb60f  Demo deployment space              2020-04-21T15:29:14.496Z
53b394b0-423e-4be0-94f7-41f92c439f54  OneAI testing deployment space     2020-03-25T17:49:26.720Z
------------------------------------  ---------------------------------  ------------------------


In [6]:
# In case you need to clear up any old spaces before this session

# client.spaces.delete('<GUID_HERE>')

View the predefined runtimes available.

In [7]:
client.runtimes.list(pre_defined=True, limit=100)

--------------------------  --------------------------  ------------------------  --------
GUID                        NAME                        CREATED                   PLATFORM
do_12.9                     do_12.9                     2020-02-12T18:10:53.504Z  do
pmml_4.3                    pmml_4.3                    2020-02-12T18:10:53.476Z  pmml
pmml_4.2.1                  pmml_4.2.1                  2020-02-12T18:10:53.451Z  pmml
pmml_4.2                    pmml_4.2                    2020-02-12T18:10:53.428Z  pmml
pmml_4.1                    pmml_4.1                    2020-02-12T18:10:53.402Z  pmml
pmml_4.0                    pmml_4.0                    2020-02-12T18:10:53.378Z  pmml
pmml_3.2                    pmml_3.2                    2020-02-12T18:10:53.351Z  pmml
pmml_3.1                    pmml_3.1                    2020-02-12T18:10:53.327Z  pmml
pmml_3.0                    pmml_3.0                    2020-02-12T18:10:53.302Z  pmml
ai-function_0.1-py3.6       ai-functi

In [8]:
client.repository.list()

------------------------------------  -----------------------  ------------------------  ---------  --------------
GUID                                  NAME                     CREATED                   FRAMEWORK  TYPE
caffe_frcnn                           caffe_frcnn              2020-02-12T18:10:51.199Z  -          Python runtime
caffe2_0.8                            caffe2_0.8               2020-02-12T18:10:51.144Z  -          Python runtime
theano_1.0                            theano_1.0               2020-02-12T18:10:51.090Z  -          Python runtime
do_12.9                               do_12.9                  2020-02-12T18:10:53.504Z  -          do runtime
hybrid_0.2                            hybrid_0.2               2020-02-12T18:10:53.227Z  -          hybrid runtime
hybrid_0.1                            hybrid_0.1               2020-02-12T18:10:53.204Z  -          hybrid runtime
torch_lua52                           torch_lua52              2020-02-12T18:10:51.351Z  -    

### Create library

Get the zip file containing the training python code is downloaded. It can be modified to use locally stored files.

In [1]:
import requests

model_content_resp = requests.get("https://github.com/calinrc/model_definitions/raw/master/libs/pytorch_onnx_v_1.1.zip",
                                  headers={"Content-Type": "application/octet-stream"})
with open("pytorch_onnx_v_1.1.zip", "wb") as f:
    f.write(model_content_resp.content)

In [10]:
meta_props = {
    client.runtimes.LibraryMetaNames.NAME: "pytorch library1",
    client.runtimes.LibraryMetaNames.FILEPATH: "./pytorch_onnx_v_1.1.zip",
    client.runtimes.LibraryMetaNames.PLATFORM: {"name": "python",
                                               "versions": ["3.6"]},
    client.runtimes.LibraryMetaNames.VERSION: "1",
    client.runtimes.LibraryMetaNames.COMMAND: "pytorch_v_1.1_mnist_onnx.py --debug-level debug"
}

library = client.runtimes.store_library(meta_props)

In [11]:
client.runtimes.get_library_details()

{'first': {'href': '/v4/libraries?space_id=07b32a16-b82e-443e-96ae-fab32da0f1b1'},
 'limit': 100,
 'resources': [{'metadata': {'guid': '575cc8fd-cb61-4c26-b1b6-a79f0679a433',
    'id': '575cc8fd-cb61-4c26-b1b6-a79f0679a433',
    'modified_at': '2020-07-10T14:41:14.952Z',
    'created_at': '2020-07-10T14:41:14.278Z',
    'href': '/v4/libraries/575cc8fd-cb61-4c26-b1b6-a79f0679a433'},
   'entity': {'name': 'pytorch library1',
    'command': 'pytorch_v_1.1_mnist_onnx.py --debug-level debug',
    'version': '1',
    'space': {'href': '/v4/spaces/07b32a16-b82e-443e-96ae-fab32da0f1b1'},
    'platform': {'name': 'python', 'versions': ['3.6']}}}]}

In [12]:
library_uid = client.runtimes.get_library_uid(library)
print("Library uid: {}".format(library_uid))

Library uid: 575cc8fd-cb61-4c26-b1b6-a79f0679a433


<a id = "train"></a>
## Train the model

### Create training

In [13]:
meta_props = {
    client.training.ConfigurationMetaNames.TRAINING_LIB_UID: library_uid,
    client.training.ConfigurationMetaNames.TRAINING_LIB_RUNTIME_UID: "pytorch-onnx_1.1-py3.6",
    client.training.ConfigurationMetaNames.COMMAND: "pytorch_v_1.1_mnist_onnx.py --debug-level debug",
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: [
        {
          "name": "training_input_data",
          "type": "fs",
          "connection": {},
          "location": {
            "path": "pytorch-mnist"
          },
          "schema": {
            "id": "id123_schema",
            "fields": [
              {
                "name": "text",
                "type": "string"
              }
            ]
          }
        }
      ],
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
        "name": "name123",
        "connection": {},
        "location": {
          "path": "/spaces/" + space_id + "/assets/trainings"
        },
        "type": "fs"
      }
}

training = client.training.run(meta_props)

In [14]:
training_uid = client.training.get_uid(training)
training_state = client.training.get_status(training_uid)['state']
print("Training uid: {}".format(training_uid))

Training uid: e474b019-66a6-4f88-9ec1-64b43b45f831


In [15]:
client.training.get_details(training_uid)

{'metadata': {'guid': 'e474b019-66a6-4f88-9ec1-64b43b45f831',
  'id': 'e474b019-66a6-4f88-9ec1-64b43b45f831',
  'modified_at': '2020-07-10T14:41:22.870Z',
  'created_at': '2020-07-10T14:41:21.465Z',
  'href': '/v4/trainings/e474b019-66a6-4f88-9ec1-64b43b45f831'},
 'entity': {'training_data_references': [{'location': {'path': 'pytorch-mnist'},
    'type': 'fs',
    'schema': {'id': 'id123_schema',
     'fields': [{'name': 'text', 'type': 'string'}]}}],
  'results_reference': {'location': {'path': '/spaces/07b32a16-b82e-443e-96ae-fab32da0f1b1/assets/trainings',
    'model': '/spaces/07b32a16-b82e-443e-96ae-fab32da0f1b1/assets/trainings/e474b019-66a6-4f88-9ec1-64b43b45f831/data/model',
    'training': '/spaces/07b32a16-b82e-443e-96ae-fab32da0f1b1/assets/trainings/e474b019-66a6-4f88-9ec1-64b43b45f831',
    'training_status': '/spaces/07b32a16-b82e-443e-96ae-fab32da0f1b1/assets/trainings/e474b019-66a6-4f88-9ec1-64b43b45f831/training-status.json',
    'logs': '/spaces/07b32a16-b82e-443e-96ae

### Monitor training

In [16]:
import time 

client.training.monitor_logs(training_uid)
while training_state == 'pending' or training_state == 'running':
    time.sleep(10)
    print("Current training state: {}".format(training_state))
    training_state = client.training.get_status(training_uid)['state']



##########################################################################

Log monitor started for training run: e474b019-66a6-4f88-9ec1-64b43b45f831

##########################################################################




-----------------
Log monitor done.
-----------------


Current training state: pending
Current training state: pending
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running
Current training state: running


<a id = "deploy"></a>
## Save and deploy the model

### Save Model to project

Note that you can save your model to your Watson Studio project by changing from the space we've been working in up to this point by using the `PROJECT_ID` variable. 

This saved model can then be promoted to the CPD deployment space from the UI (not currently possible with python client).

In [20]:
# Only use this if you wish to save model to Watson Studio project. Promotion to deployment space / deployment must then be done from UI.

project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

Unsetting the space_id ...


'SUCCESS'

Otherwise stick with the working space you've been using so far:

In [21]:
# Only use this if you need to change back to default space created in this notebook

client.set.default_space(space_id)

Unsetting the project_id ...


'SUCCESS'

In [22]:
meta_props = {
    client.repository.ModelMetaNames.NAME: "pytorch model MQ 2",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: [client.training.get_details(training_uid)["entity"]["results_reference"]],
    client.repository.ModelMetaNames.RUNTIME_UID: "pytorch-onnx_1.1-py3.6",
    client.repository.ModelMetaNames.TYPE: "pytorch-onnx_1.1"
}

model = client.repository.store_model(training_uid, meta_props)

In [23]:
model_uid = client.repository.get_model_uid(model)

In [24]:
client.repository.list_models()

------------------------------------  ------------------  ------------------------  ----------------
GUID                                  NAME                CREATED                   TYPE
7cf8a8f0-c164-406c-ba9b-430a1bffe7b2  pytorch model MQ 2  2020-05-29T22:51:55.002Z  pytorch-onnx_1.1
------------------------------------  ------------------  ------------------------  ----------------


### Create Deployment

Documentation: https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/wmls/wmls-deploy-python.html

In [25]:
client.deployments.get_details()

{'resources': []}

In [26]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "pytorch deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment = client.deployments.create(model_uid, meta_props)



#######################################################################################

Synchronous deployment creation for uid: '7cf8a8f0-c164-406c-ba9b-430a1bffe7b2' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='384a4543-a41e-45f1-a5c3-9d9f1ed7c534'
------------------------------------------------------------------------------------------------




In [27]:
deployment_uid = client.deployments.get_uid(deployment)
print("Deployment uid: {}".format(deployment_uid))

Deployment uid: 384a4543-a41e-45f1-a5c3-9d9f1ed7c534


<a id = "score"></a>
## Scoring the model

### Score deployment

In [28]:
values = [[[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0.6392157077789307,0.7568628191947937,0.5960784554481506,0.3607843220233917,0.20000001788139343,0.20000001788139343,0.20000001788139343,0.20000001788139343,0.12156863510608673,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.08235294371843338,0.874509871006012,0.9921569228172302,0.988235354423523,0.9921569228172302,0.988235354423523,0.9921569228172302,0.988235354423523,0.9921569228172302,0.7529412508010864,0.32156863808631897,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.9921569228172302,0.40000003576278687,0,0.08235294371843338,0.40000003576278687,0.24313727021217346,0.40000003576278687,0.40000003576278687,0.2392157018184662,0.7176470756530762,0.1568627506494522,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0.2392157018184662,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.20000001788139343,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.3607843220233917,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.6784313917160034,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0.9921569228172302,0.988235354423523,0.874509871006012,0.7960785031318665,0.7960785031318665,0.7960785031318665,0.32156863808631897,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0.48235297203063965,0.9960784912109375,0.9921569228172302,0.8784314393997192,0.7960785031318665,0.7960785031318665,0.874509871006012,0.9960784912109375,0.27843138575553894,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.16078431904315948,0.9529412388801575,0.9921569228172302,0.5098039507865906,0.0784313753247261,0,0,0.0784313753247261,0.9921569228172302,0.9098039865493774,0.16078431904315948,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.5960784554481506,0.9921569228172302,0.7176470756530762,0,0,0,0,0,0.5176470875740051,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.20000001788139343,0.5921568870544434,0.0784313753247261,0,0,0,0,0,0.20000001788139343,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.08235294371843338,0,0,0,0,0,0,0,0.4431372880935669,0.9921569228172302,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.32156863808631897,0.7176470756530762,0,0,0,0,0,0,0,0.7568628191947937,0.988235354423523,0.40000003576278687,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.7960785031318665,0.7176470756530762,0,0,0,0,0,0,0.08235294371843338,0.9960784912109375,0.9921569228172302,0.16078431904315948,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0.08235294371843338,0.874509871006012,0.40000003576278687,0,0,0,0,0,0.08235294371843338,0.7960785031318665,0.9921569228172302,0.5098039507865906,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.8000000715255737,0.48235297203063965,0,0,0,0,0.16078431904315948,0.6784313917160034,0.9921569228172302,0.7960785031318665,0.0784313753247261,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.6352941393852234,0.874509871006012,0.40000003576278687,0.08235294371843338,0.40000003576278687,0.6392157077789307,0.9529412388801575,0.9921569228172302,0.6705882549285889,0.0784313753247261,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.16078431904315948,0.917647123336792,0.9921569228172302,1,0.9921569228172302,1,0.6745098233222961,0.32156863808631897,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0.11764706671237946,0.5137255191802979,0.7529412508010864,0.43529415130615234,0.19607844948768616,0.03921568766236305,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]]]

meta_props = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{"values": values}]
}    
predictions = client.deployments.score(deployment_uid, meta_props)
print("Predictions returned\n{}".format(predictions))

Predictions returned
{'predictions': [{'values': [[-5.3478593826293945, -6.878868579864502, -5.569346904754639, -4.2393364906311035, -8.468282699584961, -0.08463510870933533, -3.6433980464935303, -6.80506706237793, -3.620191812515259, -5.851403713226318]]}]}


### Prediction accuracy

In [29]:
highest_log_probability = max(predictions['predictions'][0]['values'][0])
prediction = predictions['predictions'][0]['values'][0].index(highest_log_probability)
print("We predict the picture below is a {}".format(prediction))

from matplotlib import pyplot as plt 
import numpy as np

first_image = np.array(values[0][0], dtype='float')
plt.imshow(first_image, cmap='gray')
plt.show()

We predict the picture below is a 5


<Figure size 640x480 with 1 Axes>

### Interact with wider project 

In [32]:
from project_lib import Project
project = Project.access()
storage_credentials = project.get_storage_metadata()

In [33]:
project.get_name()

'CPD and WMLA demo'

## Clean up resources

In [30]:
print("Deployment deletion: {}".format(client.deployments.delete(deployment_uid)))
print("Space deletion: {}".format(client.spaces.delete(space_id)))

Deployment deletion: SUCCESS
Space deletion: SUCCESS


<a id = "summary"></a>
## Summary

Congratulations! You have learned to:

1. Download the Pytorch MNIST model
2. Create a Watson Machine Learning model by using the Pytorch model
3. Train the model by offloading work to Watson Machine Learning Acclerator
4. Save and deploy from your workspace
5. Score the model
6. Clean up